In [1]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [2]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [4]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [5]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_C_050' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_C_050
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.0016370052866505728
test_rmse: 0.0016570373834287493
test_r2: 0.6651735751146555
feature number: 3
train_rmse: 0.0016242895138231413
test_rmse: 0.0016582789522003893
test_r2: 0.6649355303742197
feature number: 4
train_rmse: 0.001606411487802367
test_rmse: 0.001644161943556466
test_r2: 0.6709653105483502
feature number: 5
train_rmse: 0.001598742846204608
test_rmse: 0.001636886558505766
test_r2: 0.6739558186643464
feature number: 6
train_rmse: 0.0015971155771804663
test_rmse: 0.0016412170022280328
test_r2: 0.6721456044017738
feature number: 7
train_rmse: 0.0015967154212758457
test_rmse: 0.0016434415473751559
test_r2: 0.6713091938946222
feature number: 8
train_rmse: 0.0015904278805986254
test_rmse: 0.0016502091424683064
test_r2: 0.6684417780699097
feature number: 9
train_rmse: 0.0015888523218812764
test_rmse: 0.0016495053186658423
test_r2: 0.6686537586117521
feature number: 10
train_rmse: 0.0015870263581329383
test_rmse

train_rmse: 0.001564467812902253
test_rmse: 0.0016880303583599841
test_r2: 0.6537386862862355
feature number: 16
train_rmse: 0.0015627071958645637
test_rmse: 0.0017005367756640667
test_r2: 0.6486094244199794
feature number: 17
train_rmse: 0.0015609985742473967
test_rmse: 0.0017099095367291849
test_r2: 0.6447482058252999
feature number: 18
train_rmse: 0.0015567789517313018
test_rmse: 0.001721712696533968
test_r2: 0.6398150390986717
feature number: 19
train_rmse: 0.001556292034511989
test_rmse: 0.0017248141521299654
test_r2: 0.6384534024603002
feature number: 20
train_rmse: 0.0015549318084325148
test_rmse: 0.0017297864233202017
test_r2: 0.6363883575663473
Test_rmse_min = 1.fea_num: 5  2.rmse: 0.001636886558505766
Test_r2_max = 1. fea_num: 5  2.r2: 0.6739558186643464
Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.0016370052866505728
test_rmse: 0.0016570373834287493
test_r2: 0.6651735751146555
feature number: 3
train_rmse: 0.0016242895138231413
test_rmse: 0.0016582789522003893
t

test_r2: 0.6684417780699097
feature number: 9
train_rmse: 0.0015888523218812764
test_rmse: 0.0016495053186658423
test_r2: 0.6686537586117521
feature number: 10
train_rmse: 0.0015870263581329383
test_rmse: 0.001648818936120755
test_r2: 0.6690174170406297
feature number: 11
train_rmse: 0.0015842919796609117
test_rmse: 0.0016544452420056866
test_r2: 0.6666829962765857
feature number: 12
train_rmse: 0.0015818154350053415
test_rmse: 0.0016579147689418608
test_r2: 0.6654615396694044
feature number: 13
train_rmse: 0.0015799866687809892
test_rmse: 0.001662849138003596
test_r2: 0.6634699982633059
feature number: 14
train_rmse: 0.0015768423209667267
test_rmse: 0.0016859924401599435
test_r2: 0.6544040463918783
feature number: 15
train_rmse: 0.001564467812902253
test_rmse: 0.0016880303583599841
test_r2: 0.6537386862862355
feature number: 16
train_rmse: 0.0015627071958645637
test_rmse: 0.0017005367756640667
test_r2: 0.6486094244199794
feature number: 17
train_rmse: 0.0015609985742473967
test_rmse: 

# 可以修改第一層的神經元數量

In [14]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(6, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [13]:
fea_num=[10,11,12] # 要測試的feature數量 神經元6

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_2/Input_C_050.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 10
test_rmse_ave: 0.005779278411922155
[0.005672187910924191, 0.003674070917371211, 0.008408764465242581, 0.0047302601345004395, 0.00535064326444741, 0.001647028530350083, 0.005187901255715656, 0.005305664435017422, 0.006302410636665031, 0.011513852568987527]


test_r2_ave: -4.1545127555862065
[-2.8624444050349074, -0.13309712110826677, -8.95403645620835, -1.7718477330018967, -1.8984490533636253, 0.6359218031539294, -3.320320855403609, -3.1982082154803484, -2.7075373909869653, -17.335108128428026]


feature number: 11
test_rmse_ave: 0.004267316139603959
[0.004433592754344192, 0.0021251355394370417, 0.0030466467664115246, 0.008779171003068882, 0.002371972232523065, 0.0017215276445857854, 0.001962896257543197, 0.002234671425450811, 0.014426545566950865, 0.0015710022057242273]


test_r2_ave: -2.5692001343876436
[-1.3597859926111426, 0.6209073756420636, -0.30670953447508165, -8.547866465921528, 0.43039619904591175, 0.6022406206597646, 0.3815176749124629, 0.25524784615219875